In [1]:
import pickle as pkl
import numpy as np
"""
for dataset in ['iwslt14', 'multi30k']:
    logs = pkl.load(open('data/{dataset}_logs.pkl'.format(dataset=dataset), 'rb'))

    num_datapoints = len(logs['sequences'])

    iterations = logs['normal_A'].keys()
    dicts = logs['sequences']
    for key in logs:
        if key == 'sequences':
            continue
        for iteration in iterations:
            for datapoint_idx in range(num_datapoints):
                dicts[datapoint_idx][(key, iteration, 'alpha')] = logs[key][iteration][datapoint_idx]['alpha']
                dicts[datapoint_idx][(key, iteration, 'beta')] = logs[key][iteration][datapoint_idx]['beta']
                dicts[datapoint_idx]['split'] = logs[key][iteration][datapoint_idx]['split']
    pkl.dump(dicts, open('data/{dataset}_logs_rz.pkl'.format(dataset=dataset), 'wb'))
    """
from utils import *

def max_acc_iter(name, metas):
    max_acc = -1.0
    max_acc_iter = None
    for key_ in metas:
        if key_[0] == name and 'test' in key_[2]:
            if metas[key_] > max_acc:
                max_acc = metas[key_]
                max_acc_iter = key_[1]
    return max_acc_iter

In [32]:
dataset = 'SMS'
dat = pkl.load(open('outputs/{dataset}_logs.pkl'.format(dataset=dataset), 'rb'))
all_dicts = dat['data']
subset = 'val'
dicts = [d for d in all_dicts if d['split'] == subset]
iterations = sorted(list(set([key[1] for key in dat['metas']])))

In [33]:
from utils import TopKMatch, TopPercentMatch, SpearmanRankCorr
metrics = [TopKMatch(k=1), TopKMatch(k=3), TopKMatch(k=5), TopPercentMatch(p=5)]
normalA_iter = max_acc_iter('normal_A', dat['metas'])
normalB_iter = max_acc_iter('normal_B', dat['metas'])
uniform_iter = max_acc_iter('uniform', dat['metas'])

In [34]:
# metrics.append(SpearmanRankCorr()) comment out because this takes a lot of time to evaluate
for iter_ in iterations:
    key1, key2 = ('normal_A', normalA_iter, 'alpha'), ('normal_B', iter_, 'beta')
    print(key1, key2)
    for metric in metrics:
        print(metric.eval_corr(dicts, key1, key2))

('normal_A', 1500, 'alpha') ('normal_B', 500, 'beta')
{'name': 'top 1 match', 'correlation': 0.06557377049180328, 'baseline': 0.07280617164898746}
{'name': 'top 3 match', 'correlation': 0.14368370298939248, 'baseline': 0.23722275795564127}
{'name': 'top 5 match', 'correlation': 0.2391513982642237, 'baseline': 0.37849566055930567}
{'name': 'top 5% match', 'correlation': 0.09450337512054002, 'baseline': 0.09209257473481196}
('normal_A', 1500, 'alpha') ('normal_B', 1500, 'beta')
{'name': 'top 1 match', 'correlation': 0.06943105110896818, 'baseline': 0.07521697203471553}
{'name': 'top 3 match', 'correlation': 0.13886210221793635, 'baseline': 0.23577627772420442}
{'name': 'top 5 match', 'correlation': 0.22806171648987464, 'baseline': 0.3780135004821601}
{'name': 'top 5% match', 'correlation': 0.0935390549662488, 'baseline': 0.0863066538090646}
('normal_A', 1500, 'alpha') ('normal_B', 3000, 'beta')
{'name': 'top 1 match', 'correlation': 0.07232401157184185, 'baseline': 0.07280617164898746}
{

In [35]:
for iter_ in iterations:
    key1, key2 = ('normal_A', normalA_iter, 'alpha'), ('uniform', iter_, 'beta')
    print(key1, key2)
    for metric in metrics:
        print(metric.eval_corr(dicts, key1, key2))

('normal_A', 1500, 'alpha') ('uniform', 500, 'beta')
{'name': 'top 1 match', 'correlation': 0.0048216007714561235, 'baseline': 0.07762777242044358}
{'name': 'top 3 match', 'correlation': 0.027965284474445518, 'baseline': 0.2266152362584378}
{'name': 'top 5 match', 'correlation': 0.06750241080038573, 'baseline': 0.3799421407907425}
{'name': 'top 5% match', 'correlation': 0.008678881388621022, 'baseline': 0.09257473481195758}
('normal_A', 1500, 'alpha') ('uniform', 1500, 'beta')
{'name': 'top 1 match', 'correlation': 0.009643201542912247, 'baseline': 0.07666345226615236}
{'name': 'top 3 match', 'correlation': 0.03326904532304725, 'baseline': 0.2304725168756027}
{'name': 'top 5 match', 'correlation': 0.08486017357762778, 'baseline': 0.39344262295081966}
{'name': 'top 5% match', 'correlation': 0.015429122468659595, 'baseline': 0.0935390549662488}
('normal_A', 1500, 'alpha') ('uniform', 3000, 'beta')
{'name': 'top 1 match', 'correlation': 0.010607521697203472, 'baseline': 0.085824493731919}

In [36]:
for iter_ in iterations:
    key1, key2 = ('uniform', iter_, 'beta'), 'logistics_beta'
    print(key1, key2)
    for metric in metrics:
        print(metric.eval_corr(dicts, key1, key2))

('uniform', 500, 'beta') logistics_beta
{'name': 'top 1 match', 'correlation': 0.3037608486017358, 'baseline': 0.11137897782063645}
{'name': 'top 3 match', 'correlation': 0.5419479267116682, 'baseline': 0.25650916104146576}
{'name': 'top 5 match', 'correlation': 0.7000964320154291, 'baseline': 0.3919961427193828}
{'name': 'top 5% match', 'correlation': 0.3254580520732883, 'baseline': 0.11089681774349083}
('uniform', 1500, 'beta') logistics_beta
{'name': 'top 1 match', 'correlation': 0.23674059787849566, 'baseline': 0.09980713596914176}
{'name': 'top 3 match', 'correlation': 0.437801350048216, 'baseline': 0.24975891996142718}
{'name': 'top 5 match', 'correlation': 0.5998071359691417, 'baseline': 0.390549662487946}
{'name': 'top 5% match', 'correlation': 0.24686595949855353, 'baseline': 0.10125361620057859}
('uniform', 3000, 'beta') logistics_beta
{'name': 'top 1 match', 'correlation': 0.23143683702989393, 'baseline': 0.09112825458052073}
{'name': 'top 3 match', 'correlation': 0.42960462